In [1]:
import datetime, pygsheets, pandas as pd


files_links_prefix = "C:/Users/foyke/iCloudDrive/Python/xyz/RSI"

#token to get access to google doc
service_account_json = files_links_prefix + '/python-304621-1e403209f05f.json'
gc = pygsheets.authorize(service_file=service_account_json)


today = datetime.datetime.now()
df_full = pd.DataFrame()
exception_list = []


In [2]:
#preparing list of all tickers to check
url = "https://docs.google.com/spreadsheets/d/1EBhcvotX9D5McF-9HHjf4hVkisyesCdLB2mgfeEX_mY/edit?usp=sharing"
gc = pygsheets.authorize(service_file=service_account_json)
sh = gc.open_by_url(url)
wks = sh[3]
all_stocks = pd.DataFrame(wks.get_all_records())

In [3]:
## Оставляем только Российские тикеры
all_stocks_ru = all_stocks.loc[all_stocks['GEO'] == 'russia']
print(len(all_stocks_ru))

244


In [4]:
#готовим даты
end_date = datetime.datetime.now()
d = datetime.timedelta(days = 365*10)
start_date = end_date - d


#спец формат даты для MOEX
start_date_mx = start_date.strftime('%Y-%m-%d')
end_date_mx = end_date.strftime('%Y-%m-%d')


In [6]:
## Функция выгрузки данных через ручку MOEX
def moex (ticker_in, start_date_mx, end_date_mx, interval):

    df = pd.DataFrame()
    global exception_list

    try:
        query = f'http://iss.moex.com/iss/engines/stock/markets/shares/securities/{ticker_in}/candles.csv?from={start_date_mx}&till={end_date_mx}&interval={interval}'
        df = pd.read_csv(query, sep=';', header=1)

        # df.rename(columns={'End': 'Date'}, inplace=True) #переименовка колонки, чтобы было всё в одном формате
        df['ticker'] = ticker_in
        # df = df.set_index('Date')
    except:
        exception_list.append(ticker_in)

    return df

Данные за 10 лет с интервалом 1 день

In [7]:
interval = 24

df_full = pd.DataFrame()

for i in range(0,len(all_stocks_ru)):
    ticker_in = all_stocks_ru['Ticker'][i]
    df = moex (ticker_in, start_date_mx, end_date_mx, interval)
    if len(df) > 0: df_full = pd.concat([df_full,df])


print(len(df_full))

df_full.to_excel('datasets/10years_data_1d_interval.xlsx',index = False)

    

115491


Данные за 10 лет с интервалом 1 час


In [8]:
interval = 60

df_full = pd.DataFrame()

for i in range(0,len(all_stocks_ru)):
    ticker_in = all_stocks_ru['Ticker'][i]
    df = moex (ticker_in, start_date_mx, end_date_mx, interval)
    if len(df) > 0: df_full = pd.concat([df_full,df])



print(len(df_full))

df_full.to_excel('datasets/10years_data_1h_interval.xlsx',index = False)

    

120362


Данные за 10 лет с интервалом 10 минут

In [9]:
interval = 10

df_full = pd.DataFrame()

for i in range(0,len(all_stocks_ru)):
    ticker_in = all_stocks_ru['Ticker'][i]
    df = moex (ticker_in, start_date_mx, end_date_mx, interval)
    if len(df) > 0: df_full = pd.concat([df_full,df])



print(len(df_full))
df_full.to_excel('datasets/10years_data_10m_interval.xlsx',index = False)

    

120512


Данные за 10 лет с интервалом 1 минута

In [10]:
interval = 1

df_full = pd.DataFrame()

for i in range(0,len(all_stocks_ru)):
    ticker_in = all_stocks_ru['Ticker'][i]
    df = moex (ticker_in, start_date_mx, end_date_mx, interval)
    if len(df) > 0: df_full = pd.concat([df_full,df])



print(len(df_full))
df_full.to_excel('datasets/10years_data_1m_interval.xlsx',index = False)

119012


In [11]:
#готовим даты для выгрузки за 30 лет
end_date = datetime.datetime.now()
d = datetime.timedelta(days = 365*30)
start_date = end_date - d


#спец формат даты для MOEX
start_date_mx = start_date.strftime('%Y-%m-%d')
end_date_mx = end_date.strftime('%Y-%m-%d')


Данные за 30 лет с интервалом 1 день


In [12]:
interval = 24

df_full = pd.DataFrame()

for i in range(0,len(all_stocks_ru)):
    ticker_in = all_stocks_ru['Ticker'][i]
    df = moex (ticker_in, start_date_mx, end_date_mx, interval)
    if len(df) > 0: df_full = pd.concat([df_full,df])



print(len(df_full))
df_full.to_excel('datasets/30years_data_1d_interval.xlsx',index = False)

    

115537


Данные за 30 лет с интервалом 1 час


In [13]:
interval = 60

df_full = pd.DataFrame()

for i in range(0,len(all_stocks_ru)):
    ticker_in = all_stocks_ru['Ticker'][i]
    df = moex (ticker_in, start_date_mx, end_date_mx, interval)
    if len(df) > 0: df_full = pd.concat([df_full,df])



print(len(df_full))
df_full.to_excel('datasets/30years_data_1h_interval.xlsx',index = False)

    

120409


Данные за 30 лет с интервалом 10 минут

In [14]:
interval = 10

df_full = pd.DataFrame()

for i in range(0,len(all_stocks_ru)):
    ticker_in = all_stocks_ru['Ticker'][i]
    df = moex (ticker_in, start_date_mx, end_date_mx, interval)
    if len(df) > 0: df_full = pd.concat([df_full,df])



print(len(df_full))
df_full.to_excel('datasets/30years_data_10m_interval.xlsx',index = False)

    

120557


Данные за 30 лет с интервалом 1 минута

In [15]:
interval = 1

df_full = pd.DataFrame()

for i in range(0,len(all_stocks_ru)):
    ticker_in = all_stocks_ru['Ticker'][i]
    df = moex (ticker_in, start_date_mx, end_date_mx, interval)
    if len(df) > 0: df_full = pd.concat([df_full,df])


print(len(df_full))
df_full.to_excel('datasets/30years_data_1m_interval.xlsx',index = False)

117058


In [18]:
exception_list = list(set(exception_list)) #дедупликация
print(len(exception_list))

7